In [8]:
import pandas as pd
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
from sklearn import preprocessing
from scipy.io.arff import loadarff
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import time
import warnings
warnings.filterwarnings("ignore")

In [9]:
# Função para chamada do gráfico interativo

def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.43.1.min.js?noext',
            },
          });
        </script>
        '''))

In [10]:
# Carrega o .arff
raw_data = loadarff('./ckplus/LBP.arff')
# Transforma o .arff em um Pandas Dataframe
df = pd.DataFrame(raw_data[0])
# Imprime o Dataframe com suas colunas
df.head()

# Com o iloc voce retira as linhas e colunas que quiser do Dataframe, no caso aqui sem as classes
X = df.iloc[:, 0:-1].values

# Aqui salvamos apenas as classes agora
y = df['class']
# Substituimos os valores binários por inteiro
y_aux = []
for i in y:
    y_aux.append(int(i.decode('ascii')[-1]))
# Novo y
y = y_aux

# Dividindo o conjunto em 80% Treino e 20% Teste.
# O parâmetro random_state = 327 define que sempre será dividido da mesma forma o conjunto.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=327)

print('Tamanho do conjunto de Treino: {}'.format(X_train.shape))
print('Tamanho do conjunto de Teste: {}'.format(X_test.shape))
scaler = preprocessing.StandardScaler()

# Escalando os dados de treinamento
X_train = scaler.fit_transform(X_train)
# Escalando os dados de teste com os dados de treinamento, visto que os dados de teste podem ser apenas 1 amostra
X_test = scaler.transform(X_test)

Tamanho do conjunto de Treino: (600, 256)
Tamanho do conjunto de Teste: (150, 256)


In [11]:
def aprendizadoAtivo(X_train, X_test, y_train, y_test):
    # Inicializar o KMeans com N centroides
    num_c = 5
    kmeans = KMeans(n_clusters = int(num_c), init = 'k-means++', random_state = 1)
    print('Numero de clusters: {}'.format(int(num_c)))
    # Executar passando como parâmetro os dados
    kmeans.fit(X_train)

    # Variavel distance recebe uma tabela de distancia de cada amostra para o centroide
    distance = kmeans.fit_transform(X_train)

    # ordered_distance.append(np.argsort(distance[:,i]))

    # agrupamento das amostras em dicionário
    cluster_x_index = {i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}

    # salvar distâncias
    cluster_distance = []
    for i in range(0, kmeans.n_clusters):
        max_len = cluster_x_index[i].size
        aux = []
        for j in range(0, max_len):
            index = cluster_x_index[i][j]
            aux.append(distance[index][i])
        cluster_distance.append(aux)

    # identificar o maior cluster 
    biggest_cluster = len(cluster_distance[0])
    for i in range(0, kmeans.n_clusters):
        if biggest_cluster < len(cluster_distance[i]):
            biggest_cluster = len(cluster_distance[i])
            
            
    # ordenar os pontos por distância decrescente, pegar apenas os índices
    sorted_index = []
    for i in range(0, kmeans.n_clusters):       
        aux_distance = np.argsort(cluster_distance[i])[::-1]
        sorted_index.append(np.argsort(cluster_distance[i])[::-1])

    # Criando as variáveis do Aprendizado Ativo fora do ciclo de iteração
    active_data = []
    active_label = []
    acc_train = []
    acc_test = []
    f1score = []
    precision = []
    recall = []
    
    iteration_count = 0
    # Selecionando amostras a partir do agrupamento
    for max_range in range(0, biggest_cluster):
        iteration_count += 1
        for cluster in range(0, kmeans.n_clusters):
            if len(sorted_index[cluster]) > max_range:
                point = max_range
                index = sorted_index[cluster][point]
                active_data.append(X_train[index])
                active_label.append(y_train[index])
        
        print('active_data len: {}'.format(len(active_data)))
        print('active_label len: {}'.format(len(active_label)))
        print('iteration: {}'.format(iteration_count))
        
        # Transforma elas em ndarray novamente
        active_data = np.asarray(active_data)
        active_label = np.asarray(active_label)

        # Neural Net
        t = time.time()
        nnet = MLPClassifier(alpha = 1, random_state = 1)
        model8 = nnet.fit(active_data, active_label)
        print('Treino do Neural Net Terminado. (Tempo de execucao: {})'.format(time.time() - t))
        print('')

        # Neural Net

        # Variavel para armazenar o tempo
        t = time.time()
        # Usando o modelo para predição das amostras de teste
        aux = nnet.predict(X_test)
        # Método para criar a matriz de confusão
        cm = confusion_matrix(y_test, aux)
        print('Matriz de Confusão:')
        print(cm)
        # Método para calcular o valor F1-Score
        f1score.append(f1_score(y_test, aux, average = 'macro'))
        # Método para calcular a Precision
        precision.append(precision_score(y_test, aux, average = 'macro'))
        # Método para calcular o Recall
        recall.append(recall_score(y_test, aux, average = 'macro'))
        # Salvando as acurácias nas listas
        acc_train.append(nnet.score(active_data, active_label))
        acc_test.append(nnet.score(X_test, y_test))
        print('Acuracia obtida com o Neural Net no Conjunto de Treinamento: {:.2f}'.format(acc_train[-1]))
        print('Acuracia obtida com o Neural Net no Conjunto de Teste: {:.2f}'.format(acc_test[-1]))
        print('Precision: {:.5f}'.format(precision[-1]))
        print('Recall: {:.5f}'.format(recall[-1]))
        print('F1-score: {:.5f}'.format(f1score[-1]))
        print('(Tempo de execucao: {:.5f})'.format(time.time() - t))
        active_data = active_data.tolist()
        active_label = active_label.tolist()

    return { 'acc_train': acc_train, 
             'acc_test': acc_test,
             'f1score': f1score,
             'precision': precision,
             'recall': recall }

results = aprendizadoAtivo(X_train, X_test, y_train, y_test)

Numero de clusters: 5
active_data len: 5
active_label len: 5
iteration: 1
Treino do Neural Net Terminado. (Tempo de execucao: 0.45449256896972656)

Matriz de Confusão:
[[ 1  5  4  9 15]
 [ 0  2  3  0  6]
 [ 0  7 15  2  2]
 [ 2  2  2  3 12]
 [ 5 15  8  4 26]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.31
Precision: 0.25023
Recall: 0.27586
F1-score: 0.25018
(Tempo de execucao: 0.01320)
active_data len: 10
active_label len: 10
iteration: 2
Treino do Neural Net Terminado. (Tempo de execucao: 0.5587146282196045)

Matriz de Confusão:
[[ 0  3  5  5 21]
 [ 0  2  5  0  4]
 [ 0  2 18  0  6]
 [ 1  0  2  1 17]
 [ 3  2 11  0 42]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.42
Precision: 0.25892
Recall: 0.32918
F1-score: 0.27579
(Tempo de execucao: 0.01437)
active_data len: 15
active_label len: 15
iteration: 3
Treino do Neural Net Terminado. (

Treino do Neural Net Terminado. (Tempo de execucao: 0.6356232166290283)

Matriz de Confusão:
[[18  2  2  7  5]
 [ 1  5  3  0  2]
 [ 0  2 23  1  0]
 [ 6  0  2  8  5]
 [ 2  4  2  2 48]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.68
Precision: 0.60290
Recall: 0.61542
F1-score: 0.60475
(Tempo de execucao: 0.01499)
active_data len: 105
active_label len: 105
iteration: 21
Treino do Neural Net Terminado. (Tempo de execucao: 0.6424851417541504)

Matriz de Confusão:
[[18  2  3  8  3]
 [ 1  4  3  0  3]
 [ 0  2 23  1  0]
 [ 8  0  1  8  4]
 [ 0  2  2  2 52]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.70
Precision: 0.60904
Recall: 0.61103
F1-score: 0.60618
(Tempo de execucao: 0.01475)
active_data len: 110
active_label len: 110
iteration: 22
Treino do Neural Net Terminado. (Tempo de execucao: 0.6475982666015625)

Matriz de Confusão:
[[18  2  

Treino do Neural Net Terminado. (Tempo de execucao: 0.8027784824371338)

Matriz de Confusão:
[[22  2  3  3  4]
 [ 1  5  3  0  2]
 [ 0  1 24  1  0]
 [ 6  0  1 10  4]
 [ 0  0  2  2 54]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.77
Precision: 0.71593
Recall: 0.68638
F1-score: 0.69281
(Tempo de execucao: 0.01476)
active_data len: 200
active_label len: 200
iteration: 40
Treino do Neural Net Terminado. (Tempo de execucao: 0.8017077445983887)

Matriz de Confusão:
[[22  2  3  3  4]
 [ 1  5  3  0  2]
 [ 0  1 24  1  0]
 [ 6  0  1 10  4]
 [ 0  0  2  1 55]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.77
Precision: 0.72474
Recall: 0.68983
F1-score: 0.69763
(Tempo de execucao: 0.01474)
active_data len: 205
active_label len: 205
iteration: 41
Treino do Neural Net Terminado. (Tempo de execucao: 1.1677343845367432)

Matriz de Confusão:
[[23  1  

Treino do Neural Net Terminado. (Tempo de execucao: 1.4144246578216553)

Matriz de Confusão:
[[24  1  2  4  3]
 [ 0  7  2  1  1]
 [ 0  1 24  1  0]
 [ 4  1  1 12  3]
 [ 0  0  1  2 55]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.81
Precision: 0.76885
Recall: 0.75701
F1-score: 0.76001
(Tempo de execucao: 0.01572)
active_data len: 295
active_label len: 295
iteration: 59
Treino do Neural Net Terminado. (Tempo de execucao: 1.5136966705322266)

Matriz de Confusão:
[[25  1  1  3  4]
 [ 0  7  2  1  1]
 [ 0  1 24  1  0]
 [ 4  0  1 12  4]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.83
Precision: 0.79913
Recall: 0.76634
F1-score: 0.77847
(Tempo de execucao: 0.01519)
active_data len: 300
active_label len: 300
iteration: 60
Treino do Neural Net Terminado. (Tempo de execucao: 1.4449944496154785)

Matriz de Confusão:
[[25  1  

Treino do Neural Net Terminado. (Tempo de execucao: 1.6267297267913818)

Matriz de Confusão:
[[26  1  1  3  3]
 [ 0  9  1  1  0]
 [ 0  0 26  0  0]
 [ 6  1  0 10  4]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.85
Precision: 0.81656
Recall: 0.80492
F1-score: 0.80654
(Tempo de execucao: 0.01950)
active_data len: 387
active_label len: 387
iteration: 78
Treino do Neural Net Terminado. (Tempo de execucao: 1.79707932472229)

Matriz de Confusão:
[[26  1  1  3  3]
 [ 0  9  1  1  0]
 [ 0  0 26  0  0]
 [ 6  1  0 10  4]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.85
Precision: 0.81656
Recall: 0.80492
F1-score: 0.80654
(Tempo de execucao: 0.01623)
active_data len: 390
active_label len: 390
iteration: 79
Treino do Neural Net Terminado. (Tempo de execucao: 1.5827679634094238)

Matriz de Confusão:
[[26  1  1 

...

Treino do Neural Net Terminado. (Tempo de execucao: 2.1793711185455322)

Matriz de Confusão:
[[25  2  0  3  4]
 [ 0  8  1  1  1]
 [ 0  0 26  0  0]
 [ 4  1  1 11  4]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.84
Precision: 0.80699
Recall: 0.79038
F1-score: 0.79431
(Tempo de execucao: 0.01679)
active_data len: 509
active_label len: 509
iteration: 135
Treino do Neural Net Terminado. (Tempo de execucao: 2.2257931232452393)

Matriz de Confusão:
[[25  2  0  3  4]
 [ 0  8  1  1  1]
 [ 0  0 26  0  0]
 [ 4  1  1 11  4]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.84
Precision: 0.80699
Recall: 0.79038
F1-score: 0.79431
(Tempo de execucao: 0.01697)
active_data len: 511
active_label len: 511
iteration: 136
Treino do Neural Net Terminado. (Tempo de execucao: 2.246446132659912)

Matriz de Confusão:
[[25  2 

Treino do Neural Net Terminado. (Tempo de execucao: 2.5553553104400635)

Matriz de Confusão:
[[25  2  0  3  4]
 [ 0  8  1  1  1]
 [ 0  0 26  0  0]
 [ 5  1  1 11  3]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.84
Precision: 0.80393
Recall: 0.79038
F1-score: 0.79332
(Tempo de execucao: 0.01611)
active_data len: 537
active_label len: 537
iteration: 154
Treino do Neural Net Terminado. (Tempo de execucao: 2.145613431930542)

Matriz de Confusão:
[[25  2  0  3  4]
 [ 0  8  1  1  1]
 [ 0  0 26  0  0]
 [ 5  1  1 11  3]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.84
Precision: 0.80393
Recall: 0.79038
F1-score: 0.79332
(Tempo de execucao: 0.01632)
active_data len: 538
active_label len: 538
iteration: 155
Treino do Neural Net Terminado. (Tempo de execucao: 2.1586294174194336)

Matriz de Confusão:
[[25  2 

Treino do Neural Net Terminado. (Tempo de execucao: 2.9504692554473877)

Matriz de Confusão:
[[26  2  0  2  4]
 [ 1  8  1  1  0]
 [ 0  0 26  0  0]
 [ 5  1  1 11  3]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.85
Precision: 0.81171
Recall: 0.79626
F1-score: 0.79947
(Tempo de execucao: 0.01665)
active_data len: 556
active_label len: 556
iteration: 173
Treino do Neural Net Terminado. (Tempo de execucao: 2.2705461978912354)

Matriz de Confusão:
[[26  2  0  2  4]
 [ 1  8  1  1  0]
 [ 0  0 26  0  0]
 [ 5  1  1 11  3]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.85
Precision: 0.81171
Recall: 0.79626
F1-score: 0.79947
(Tempo de execucao: 0.01682)
active_data len: 557
active_label len: 557
iteration: 174
Treino do Neural Net Terminado. (Tempo de execucao: 2.2679636478424072)

Matriz de Confusão:
[[26  2

Treino do Neural Net Terminado. (Tempo de execucao: 2.3116297721862793)

Matriz de Confusão:
[[27  1  0  2  4]
 [ 0  9  1  1  0]
 [ 0  0 26  0  0]
 [ 5  1  1 11  3]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.86
Precision: 0.83614
Recall: 0.82033
F1-score: 0.82371
(Tempo de execucao: 0.01680)
active_data len: 575
active_label len: 575
iteration: 192
Treino do Neural Net Terminado. (Tempo de execucao: 2.2885570526123047)

Matriz de Confusão:
[[26  2  0  2  4]
 [ 0  9  1  1  0]
 [ 0  0 26  0  0]
 [ 5  1  1 11  3]
 [ 0  0  1  1 56]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.85
Precision: 0.82150
Recall: 0.81444
F1-score: 0.81296
(Tempo de execucao: 0.01694)
active_data len: 576
active_label len: 576
iteration: 193
Treino do Neural Net Terminado. (Tempo de execucao: 2.3036415576934814)

Matriz de Confusão:
[[26  2

Treino do Neural Net Terminado. (Tempo de execucao: 2.3904969692230225)

Matriz de Confusão:
[[26  2  0  2  4]
 [ 0  8  1  1  1]
 [ 0  0 26  0  0]
 [ 5  1  1 12  2]
 [ 1  0  2  1 54]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.84
Precision: 0.80834
Recall: 0.79889
F1-score: 0.79999
(Tempo de execucao: 0.01673)
active_data len: 594
active_label len: 594
iteration: 211
Treino do Neural Net Terminado. (Tempo de execucao: 2.8616936206817627)

Matriz de Confusão:
[[26  2  0  2  4]
 [ 0  8  1  1  1]
 [ 0  0 26  0  0]
 [ 5  1  1 12  2]
 [ 1  0  2  1 54]]
Acuracia obtida com o Neural Net no Conjunto de Treinamento: 1.00
Acuracia obtida com o Neural Net no Conjunto de Teste: 0.84
Precision: 0.80834
Recall: 0.79889
F1-score: 0.79999
(Tempo de execucao: 0.01748)
active_data len: 595
active_label len: 595
iteration: 212
Treino do Neural Net Terminado. (Tempo de execucao: 2.4820308685302734)

Matriz de Confusão:
[[26  2

In [12]:
print(len(results['acc_test']))

217


In [13]:
iteracoes = []
for i in range(0, len(results['acc_test'])):
    iteracoes.append(i + 1)

# Criando valores do eixo X
eixo_x = ['Neural Net']

# Plotando os gráficos

dados_nnet = go.Scatter(
    # Eixo x recebe a iteração
    x = iteracoes,
    # Eixo y recebe os valores de acurácia
    y = results['acc_test'],
    # Define o nome
    name = 'Neural Net',
    mode = 'lines+markers'
)

# Alterando configurações de Layout do Gráfico
layout = go.Layout(
    # Define Título
    title = 'Acurácia dos Classificadores conforme as Iterações',
    # Define o nome do eixo X
    xaxis = {'title': 'Iterações'},
    # Define o nome do eixo Y
    yaxis = {'title':'Acurácia'},
    # Define a cor da borda e contorno do gráfico
    paper_bgcolor='rgba(245, 246, 249, 1)',
    # Define a cor do fundo do gráfico
    plot_bgcolor='rgba(245, 246, 249, 1)'
)

# Plotando
data = [dados_nnet]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
